<a href="https://colab.research.google.com/github/shardulburde/Exotic-Options-Pricing/blob/main/DigitalOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#Reading Excel Data
#Excel sheet link in next comment
#https://drive.google.com/file/d/1UXq6kIWAaGUmzk41B4ReCg5L_k_vBtas/view?usp=sharing

import pandas as pd
xls = pd.ExcelFile('/content/drive/MyDrive/IndusInd/Exotic Options.xlsx')
df = pd.read_excel(xls, 'DigitalData')
df1 = pd.read_excel(xls, 'MktData')

In [39]:
df

,Unnamed: 0,Call,Unnamed: 2
0,Strike,74,NaN
1,Spot,73.32,NaN
2,Fixed Payoff,75,NaN
3,Start Date,2021-06-21 00:00:00,NaN
4,End Date,2022-06-21 00:00:00,NaN
5,Tenor(days),365,NaN
6,Tenor(years),1,NaN
7,INR Rate(%),0.0425,NaN
8,USD Rate(%),0.002,NaN
9,Forward,76.3504,NaN


In [29]:
#Interpolating Market Vol Data
from scipy import interpolate

def atm(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,4,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def rr25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,7,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

def fly25d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,10,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
      
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)


def rr10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,13,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)

  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)
      

def fly10d(tenor):

  x_points=pd.DataFrame([])   
  y_points=pd.DataFrame([])  

  for i in range(19):
    x=pd.DataFrame([df1._get_value(i+1,1,True)])
    y=pd.DataFrame([df1._get_value(i+1,16,True)])
    x_points=x_points.append(x, ignore_index = True)
    y_points=y_points.append(y, ignore_index = True)
    
  tck = interpolate.splrep(x_points, y_points)
  return interpolate.splev(tenor, tck)

In [30]:
#Making Call Delta functions that return vol for given tenor from mkt data

def d10(tenor):
  vol=atm(tenor)+fly10d(tenor)+rr10d(tenor)/2

  return vol

def d25(tenor):
  vol=atm(tenor)+fly25d(tenor)+rr25d(tenor)/2

  return vol

def d75(tenor):
  vol=atm(tenor)+fly25d(tenor)-rr25d(tenor)/2

  return vol

def d90(tenor):
  vol=atm(tenor)+fly10d(tenor)-rr10d(tenor)/2

  return vol

In [31]:
#Interpolation for vol for tenor b/w 0.1, 0.25, 0.5, 0.75 and 0.9 call delta

from scipy import interpolate

def vol(delta,tenor):

  y1=d10(tenor)
  y2=d25(tenor)
  y3=atm(tenor)
  y4=d75(tenor)
  y5=d90(tenor)

  x_points = [0.1, 0.25, 0.5, 0.75, 0.9]
  y_points = [y1,y2,y3,y4,y5]

  tck = interpolate.splrep(x_points, y_points)
  return float(interpolate.splev(delta, tck))

vol(0.1,1)

0.07722499999999997

In [33]:
#Making Vol Lookup Function

import numpy as np
import math
import scipy
from scipy.stats import norm

#Vol for
def VolDig():
  
  tenor=df._get_value(6,1,True)
  rate=df._get_value(7,1,True)-df._get_value(8,1,True)
  strike=df._get_value(0,1,True)
  spot=df._get_value(1,1,True)

  
  v = atm(tenor)
    
  for i in range(4):
    d=(np.log(spot/strike)+(rate-v*v/2)*tenor)/(v*math.sqrt(tenor))
    delta=scipy.stats.norm.cdf(d) 
    v=vol(delta,tenor) 

  return v

x=VolDig()
print(x)
  

0.05820287330282497


In [36]:
import numpy as np
import math

def Dig(strike, spot, tenor, INRr, USDr, vol, t, numsim, fix):
  
  
  rate = INRr-USDr
  
  sum = 0.0
  
  if vol==0:
    vol=Vol()
  else:
    vol=vol

  n=round(tenor*365/t)

    
  for i in range(numsim):

    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

   
  if fwd > strike:
    payoff= fix
  
  else:
    payoff=0

  sum = sum + payoff

  Dig_Call=100*(sum / numsim) * math.exp(-rate * tenor)

 
  #PUT

  sum = 0
  
  for i in range(numsim):
  
    fwd=spot
    
    for j in range(n):
      fwd = fwd * math.exp((rate - (vol * vol) / 2) * (t/365) + vol * math.sqrt(t/365) * np.random.normal(loc=0,scale=1.0,size=None))

   
  if fwd < strike:
    payoff = fix
  
  else:
    payoff=0

  sum=sum+payoff

  Dig_Put=100*(sum / numsim) * math.exp(-rate * tenor)


         
  return Dig_Call,Dig_Put

In [45]:
print('Call and Put Price of Digital Option is',Dig(df._get_value(0,1,True),df._get_value(1,1,True),df._get_value(6,1,True),df._get_value(7,1,True),df._get_value(8,1,True),df._get_value(10,1,True),df._get_value(11,1,True),int(df._get_value(12,1,True)),df._get_value(2,1,True)))

Call and Put Price of Digital Option is (7.202318733835598, 0.0)
